# GANs for Tabular Synthetic Data Generation

Typically GANs are used to generate images. However, we can also generate tabular data from a GAN. In this part, we will use the Python tabgan utility to create fake data from tabular data. Specifically, we will use the Auto MPG dataset to train a GAN to generate fake cars.  [Cite:ashrapov2020tabular](https://arxiv.org/pdf/2010.00638.pdf)

## Installing Tabgan

Pytorch is the foundation of the tabgan neural network utility. The following code installs the needed software to run tabgan in Google Colab.

In [1]:
# HIDE OUTPUT
CMD = "wget https://raw.githubusercontent.com/Diyago/"\
  "GAN-for-tabular-data/master/requirements.txt"

!{CMD}
!pip install -r requirements.txt
!pip install tabgan

'wget' is not recognized as an internal or external command,
operable program or batch file.


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


^C


INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   -------------- ------------------------- 0.5/1.4 MB 4.2 MB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 3.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   -- ------------------------------------- 0.8/11.0 MB 4.2 MB/s eta 0:00:03
   ----- ---------------------------------- 1.6/11.0 MB 4.0 MB/s eta 0:00:03
   -------- ------------------------------- 2.4/11.0 MB 3.9 MB/s eta 0:00:03
   ----------- ---------------------------- 3.1/11.0 MB 3.9 MB/s eta 0:00:03
   -------------- ------------------------- 3.9/11.0 MB 3.9 MB/s eta 0:00:02
   ----------------- ---------------------- 4.7/11.0 MB 3.9 MB/s eta 0:00:02
   ------------------- -------------------- 5.5/11.0 MB 3.9 MB/s eta 0:00:02
   ------------------

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics


# Load your dataset
rat_df = pd.read_csv('dataset/ratings_small.csv')

# Split into training and test sets
df_x_train, df_x_test, df_y_train, df_y_test = train_test_split(
    rat_df.drop("rating", axis=1),
    rat_df["rating"],
    test_size=0.20,
    #shuffle=False,
    random_state=42,
)

# Create dataframe versions for tabular GAN
df_x_test, df_y_test = df_x_test.reset_index(drop=True), df_y_test.reset_index(drop=True)
df_y_train = pd.DataFrame(df_y_train)
df_y_test = pd.DataFrame(df_y_test)

## Training a GAN for Auto Ratings Dataset


In [9]:
from tabgan.sampler import GANGenerator
import pandas as pd

gen_x, gen_y = GANGenerator(gen_x_times=1.1, cat_cols=None,
           bot_filter_quantile=0.001, top_filter_quantile=0.999, \
              is_post_process=True,
           adversarial_model_params={
               "metrics": "rmse", "max_depth": 2, "max_bin": 100,
               "learning_rate": 0.02, "random_state": \
                42, "n_estimators": 500,
           }, pregeneration_frac=2, only_generated_data=False,\
           gen_params = {"batch_size": 500, "patience": 25, \
          "epochs" : 500,}).generate_data_pipe(df_x_train, df_y_train,\
          df_x_test, deep_copy=True, only_adversarial=False, \
          use_adversarial=True)

Fitting CTGAN transformers for each column:   0%|          | 0/4 [00:00<?, ?it/s]

Training CTGAN, epochs::   0%|          | 0/500 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 16001, number of negative: 16000
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001147 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 300
[LightGBM] [Info] Number of data points in the train set: 32001, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500016 -> initscore=0.000062
[LightGBM] [Info] Start training from score 0.000062
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -

## Evaluating the GAN Results

If we display the results, we can see that the GAN-generated data looks similar to the original. Some values, typically whole numbers in the original data, have fractional values in the synthetic data.

In [10]:
gen_x


,userId,movieId,timestamp
0,510,148193,833723329
1,510,137102,874400880
2,512,131157,853903604
3,511,147099,885133879
4,512,145885,846350876
...,...,...,...
177252,624,142997,1476465026
177253,624,139642,1465727277
177254,624,140725,1462187665
177255,624,144714,1474972061


In [11]:
gen_y

0         4.563794
1         1.746313
2         0.807055
3         5.057099
4         1.048710
            ...   
177252    2.500000
177253    3.000000
177254    3.000000
177255    1.500000
177256    1.500000
Name: rating, Length: 177257, dtype: float64

In [21]:
# Concatenate gen_x and gen_y along the columns
combined_genrated_data = pd.concat([gen_x, gen_y], axis=1)

# Save to CSV
combined_genrated_data.to_csv('Genrated ratings Data.csv', index=False)

In [22]:
combined_genrated_data

,userId,movieId,timestamp,rating
0,510,133730,849588178,2.097580
1,510,148672,846457615,3.033221
2,513,146590,851162656,2.529508
3,510,132772,846242746,2.091977
4,514,146968,833662076,0.333745
...,...,...,...,...
182987,624,142536,1473962172,2.000000
182988,624,140725,1462187665,3.000000
182989,624,144714,1474972061,1.500000
182990,624,143257,1461356507,2.500000


# It always seems impossible until it's done